## 문장을 판별하는 LSTM 구현

- IMDB 영화 추천 데이터를 이용하여 영화평을 분석해 영화 평점 정보 예측
- IMDB는 25,000건의 영화평과 이진화된 영화 평점 정보(추천=1, 미추천=0)를 담고 있다
- 평점 정보는 별점(star points)이 많은 경우는 긍정, 그렇지 않은 경우는 부정으로 나눠짐

In [2]:
# 라이브러리 임포트
from __future__ import print_function
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models

Using TensorFlow backend.
/Users/leesu/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


데이터셋에 들어 있는 문장들은 길이가 다르므로 pad_sequences() 함수를 사용해 동일하게 만듬. maxlen 보다 작으면 부족한 부분은 0으로 채우고 길면 잘라낸다.

In [5]:
# 데이터 준비
class Data:
    def __init__(self, max_features=20000, maxlen=80):
        (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
        x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
        x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

layers 는 인공 신경망의 계층을 만드는 서브패키지이다.
- Dense : 완전 연결 계층을 만드는 클래스
- Embedding : 단어를 의미 벡터로 바꾸는 계층에 대한 클래스
- LSTM : LSTM 게층을 만드는 클래스

In [6]:
# 모델링
class RNN_LSTM(models.Model):
    def __init__(self, max_features, maxlen):
        x = layers.Input((maxlen,))  # 입력 각 샘플은 80개의 원소로 된 1차원 배열 
        h = layers.Embedding(max_features, 128)(x) # 각 단어가 128의 길이를 가지는 벡트로 바뀜 80x128
        h = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(h) # 노드 128개로 구성된 LSTM 계층
        y = layers.Dense(1, activation='sigmoid')(h)
        super().__init__(x, y)
        
        self.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# 학습 및 성능 평가
class Machine:
    def __init__(self, max_features=20000, maxlen=80):
        self.data = Data(max_features, maxlen)
        self.model = RNN_LSTM(max_features, maxlen)
        
    def run(self, epochs=3, batch_size=32):
        data = self.data
        model = self.model
        print("trainig stage")
        print("=="*10)
        model.fit(data.x_train, data.y_train, batch_size=batch_size, 
                  epochs=epochs, validation_data=(data.x_test, data.y_test))
        
        score, acc = model.evaluate(data.x_test, data.y_test, batch_size=batch_size)
        print("Test performance: accuracy={0}, loss={1}".format(acc, score))

In [8]:
def main():
    m = Machine()
    m.run()
    
if __name__ == '__main__':
    main()

17465344/17464789 [==============================] - 16s 1us/step
trainig stage
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 107s 4ms/step - loss: 0.4616 - acc: 0.7817 - val_loss: 0.4236 - val_acc: 0.8117
Epoch 2/3
25000/25000 [==============================] - 108s 4ms/step - loss: 0.2996 - acc: 0.8793 - val_loss: 0.3950 - val_acc: 0.8326
Epoch 3/3
25000/25000 [==============================] - 18s 724us/step
Test performance: accuracy=0.82384, loss=0.4148980760264397
